In [13]:
import numpy as np
from random import shuffle
from PIL import Image, ImageDraw

X = 256
Y = 256
first = 0, 0
Xmax, Ymax = 2048, 4096
twocolors = "purple", "#FF7F50"
RANDOM = True
MORE_COLORS = True
ISOLATED_PREDEFINED = False
PREDEFINED_COLOR = "black"
SHUFFLE_COLORS = False
LONE_SQUARES_LONE_COLOR = False

if RANDOM:
    x, y = np.random.randint(2, size=X), np.random.randint(2, size=Y)
else:
    x = np.zeros(X, dtype=int)
    y = np.zeros(Y, dtype=int)
    x[0], y[0] = first[0], first[1]
    old, new = 0, 1
    for i in range(1, X):
        if i == new:
            old = new
            new <<= 1
        x[i] = not x[i - old]
    old, new = 0, 1
    for i in range(1, Y):
        if i == new:
            old = new
            new <<= 1
        y[i] = not y[i - old]

### THUE-MORSExTHUE-MORSE MAP
# SQUARE_SIZE = max(2, min(Xmax // X, Ymax // Y))
# field = np.zeros((X, Y), dtype=int)
# for i in range(X):
#     for j in range(Y):
#         field[i, j] = x[i] ^ y[j]
# img = Image.new("1", (SQUARE_SIZE * X, SQUARE_SIZE * Y))
# imgdraw = ImageDraw.Draw(img)
# for i in range(X):
#     for j in range(Y):
#         if field[i, j]:
#             imgdraw.rectangle([i*SQUARE_SIZE, j*SQUARE_SIZE, (i+1)*SQUARE_SIZE-1, (j+1)*SQUARE_SIZE-1], fill="white")
# img.show()
# img.save("Thue_Morse_" + str(X) + "x" + str(Y) + ".png")

### DASHES THUE-MORSE
SQUARE_SIZE = max(2, min(Xmax // (X+1), Ymax // (Y+1)))
field = np.zeros((X+1, Y+1), dtype=int)
for j in range(Y):
    field[0, j+1] = field[0, j] ^ y[j]
zero = np.zeros((Y+1), dtype=int)
ones = np.zeros((Y+1), dtype=int)
for i in range(Y+1):
    zero[i] = i % 2
    ones[i] = not (i % 2)
for i in range(X):
    if x[i]:
        field[i+1] = np.logical_xor(field[i], ones)
    else:
        field[i+1] = np.logical_xor(field[i], zero)

if MORE_COLORS:
    # PLUS  - ones
    # MINUS - zeros
    UNDEF = (X+2) * (Y+2)
    colorfield = (field * 2 - 1) * UNDEF
    # Lone squares
    if LONE_SQUARES_LONE_COLOR:
        if field[0,0] != field[0,1] and field[0,0] != field[1,0]:
            colorfield[0,0] = 0
        if field[X,0] != field[X,1] and field[X,0] != field[X-1,0]:
            colorfield[X,0] = 0
        if field[0,Y] != field[0,Y-1] and field[0,Y] != field[1,Y]:
            colorfield[0,Y] = 0
        if field[X,Y] != field[X,Y-1] and field[X,Y] != field[X-1,Y]:
            colorfield[X,Y] = 0
        for i in range(1, X):
            if field[i, 0] != field[i, 1] and field[i, 0] != field[i-1, 0] and field[i, 0] != field[i+1, 0]:
                colorfield[i, 0] = 0
            if field[i, Y] != field[i, Y-1] and field[i, Y] != field[i-1, Y] and field[i, Y] != field[i+1, Y]:
                colorfield[i, Y] = 0
        for j in range(1, Y):
            if field[0, j] != field[1, j] and field[0, j] != field[0, j-1] and field[0, j] != field[0, j+1]:
                colorfield[0, j] = 0
            if field[X, j] != field[X-1, j] and field[X, j] != field[X, j-1] and field[X, j] != field[X, j+1]:
                colorfield[X, j] = 0
        for i in range(1, X):
            for j in range(1, Y):
                if field[i, j] != field[i-1, j] and field[i, j] != field[i+1, j] and field[i, j] != field[i, j-1] and field[i, j] != field[i, j+1]:
                    colorfield[i, j] = 0
    # Other regions
    curcolor0 = 0
    curcolor1 = 0
    unvisited = np.ones((X+1, Y+1), dtype=bool)
    out_stack = [(np.random.randint(X+1), np.random.randint(Y+1))]
    unvisited[out_stack[0][0], out_stack[0][1]] = False
    while out_stack:
        i, j = out_stack.pop()
        if colorfield[i, j] == UNDEF:
            # ones
            curcolor1 += 1
            colorfield[i, j] = curcolor1
            stack = [(i, j)]
            while stack:
                ii, jj = stack.pop()
                if ii and colorfield[ii-1, jj] == UNDEF:
                    colorfield[ii-1, jj] = curcolor1
                    stack.append((ii-1, jj))
                if jj and colorfield[ii, jj-1] == UNDEF:
                    colorfield[ii, jj-1] = curcolor1
                    stack.append((ii, jj-1))
                if ii < X and colorfield[ii+1, jj] == UNDEF:
                    colorfield[ii+1, jj] = curcolor1
                    stack.append((ii+1, jj))
                if jj < Y and colorfield[ii, jj+1] == UNDEF:
                    colorfield[ii, jj+1] = curcolor1
                    stack.append((ii, jj+1))
        elif colorfield[i, j] == -UNDEF:
            # zeros
            curcolor0 -= 1
            colorfield[i, j] = curcolor0
            stack = [(i, j)]
            while stack:
                ii, jj = stack.pop()
                if ii and colorfield[ii-1, jj] == -UNDEF:
                    colorfield[ii-1, jj] = curcolor0
                    stack.append((ii-1, jj))
                if jj and colorfield[ii, jj-1] == -UNDEF:
                    colorfield[ii, jj-1] = curcolor0
                    stack.append((ii, jj-1))
                if ii < X and colorfield[ii+1, jj] == -UNDEF:
                    colorfield[ii+1, jj] = curcolor0
                    stack.append((ii+1, jj))
                if jj < Y and colorfield[ii, jj+1] == -UNDEF:
                    colorfield[ii, jj+1] = curcolor0
                    stack.append((ii, jj+1))
        new_cells = []
        if i and unvisited[i-1, j]:
            unvisited[i-1, j] = False
            new_cells.append((i-1, j))
        if j and unvisited[i, j-1]:
            unvisited[i, j-1] = False
            new_cells.append((i, j-1))
        if i < X and unvisited[i+1, j]:
            unvisited[i+1, j] = False
            new_cells.append((i+1, j))
        if j < Y and unvisited[i, j+1]:
            unvisited[i, j+1] = False
            new_cells.append((i, j+1))
        shuffle(new_cells)
        out_stack += new_cells
    num_colors = curcolor1 - curcolor0
    if not ISOLATED_PREDEFINED:
        num_colors += 1
    shift = np.random.randint(1530)
    colors = [0] * num_colors
    colors[0] = shift
    inc = 1530 / num_colors
    for i in range(1, num_colors):
        colors[i] = (shift + round(i * inc)) % 1530
    for i in range(num_colors):
        hextuple = (0, 0, 0)
        if colors[i] < 255:
            hextuple = (255, colors[i], 0)
        elif colors[i] < 510:
            hextuple = (510 - colors[i], 255, 0)
        elif colors[i] < 765:
            hextuple = (0, 255, colors[i] - 510)
        elif colors[i] < 1020:
            hextuple = (0, 1020 - colors[i], 255)
        elif colors[i] < 1275:
            hextuple = (colors[i] - 1020, 0, 255)
        else:
            hextuple = (255, 0, 1530 - colors[i])
        assert (255 <= sum(hextuple) <= 510), "something went wrong with colors..."
        colors[i] = "#"
        if hextuple[0] < 16:
            colors[i] = colors[i] + "0" + hex(hextuple[0])[2:]
        else:
            colors[i] = colors[i] + hex(hextuple[0])[2:]
        if hextuple[1] < 16:
            colors[i] = colors[i] + "0" + hex(hextuple[1])[2:]
        else:
            colors[i] = colors[i] + hex(hextuple[1])[2:]
        if hextuple[2] < 16:
            colors[i] = colors[i] + "0" + hex(hextuple[2])[2:]
        else:
            colors[i] = colors[i] + hex(hextuple[2])[2:]
    if SHUFFLE_COLORS:
        shuffle(colors)
    if ISOLATED_PREDEFINED:
        colors.insert(0, PREDEFINED_COLOR)
    img = Image.new("RGB", (SQUARE_SIZE * (X+1), SQUARE_SIZE * (Y+1)))
    imgdraw = ImageDraw.Draw(img)
    for i in range(X+1):
        for j in range(Y+1):
            imgdraw.rectangle([i*SQUARE_SIZE, j*SQUARE_SIZE, (i+1)*SQUARE_SIZE-1, (j+1)*SQUARE_SIZE-1], fill=colors[colorfield[i, j]])
    img.show()
    img.save("Dashes_Thue_Morse_" + str(X) + "x" + str(Y) + ".png")

else:
    img = Image.new("RGB", (SQUARE_SIZE * (X+1), SQUARE_SIZE * (Y+1)))
    imgdraw = ImageDraw.Draw(img)
    for i in range(X+1):
        for j in range(Y+1):
            imgdraw.rectangle([i*SQUARE_SIZE, j*SQUARE_SIZE, (i+1)*SQUARE_SIZE-1, (j+1)*SQUARE_SIZE-1], fill=twocolors[field[i, j]])
    img.show()
    img.save("Dashes_Thue_Morse_" + str(X) + "x" + str(Y) + ".png")